In [1]:
import ROOT as r
import math as m
%jsroot on

Welcome to JupyROOT 6.18/04


In [2]:
folder = '/disk/lhcb_data2/RLcMuonic2016/MC_full_new/'
mcsamples = ['Lb_Lcmunu','Lb_Lctaunu','Lb_LcDs','Lb_Lc2593munu','Lb_Lc2593taunu','Lb_Lc2593Ds',
             'Lb_Lc2625munu','Lb_Lc2625taunu','Lb_Lc2625Ds']
polarities = ['MagUp','MagDown']

In [31]:
def PutTogetherPolarityHistos(h):
    h_new = {}
    for mcsample in mcsamples:
        if(h[mcsample]['MagUp']):
            h_new[mcsample] = h[mcsample]['MagUp']
            if(h[mcsample]['MagDown']):
                h_new[mcsample].Add(h[mcsample]['MagDown'])
                h_new[mcsample].SetTitle(mcsample)
                h_new[mcsample].SetDirectory(0)
    return h_new

In [4]:
def GetTemplateHistos(cut):
    h_q2 = {mcsample: {polarity: r.TH1F('h_q2_'+mcsample+'_'+polarity,'',40,-2,12) for polarity in polarities} for mcsample in mcsamples}
    h_MM2 = {mcsample: {polarity: r.TH1F('h_Mmiss2_'+mcsample+'_'+polarity,'',100,-2,12) for polarity in polarities} for mcsample in mcsamples}
    h_El = {mcsample: {polarity: r.TH1F('h_El2_'+mcsample+'_'+polarity,'',100,0,2600) for polarity in polarities} for mcsample in mcsamples}
    for mcsample in mcsamples:
        for polarity in polarities:
            startf = r.TFile(folder+mcsample+'_'+polarity+'_full.root','READ')
            startt = startf.Get('tupleout/DecayTree')
            fPID = r.TFile(folder+mcsample+'_'+polarity+'_full_PIDCalib.root','READ')
            tPID = fPID.Get('tupleout/DecayTree')
            preself = r.TFile(folder+mcsample+'_'+polarity+'_full_preselectionVars.root','READ')
            preselt = preself.Get('DecayTree')
            startt.AddFriend(preselt)
            startt.AddFriend(tPID)
            name = mcsample+'_'+polarity
            startt.Draw('FitVar_q2_mLc/1.E6>>q2_'+name+'(40,-2,14)','w_LbCorr*Event_PIDCalibEffWeight*(FitVar_q2_mLc/1.E6>-2 && FitVar_q2_mLc/1.E6<14'+cut+')')
            h_q2[mcsample][polarity] = r.gPad.GetPrimitive('q2_'+name)
            startt.Draw('FitVar_Mmiss2_mLc/1.E6>>MM2_'+name+'(100,-2,14)','w_LbCorr*Event_PIDCalibEffWeight*(FitVar_Mmiss2_mLc/1.E6>-2 && FitVar_Mmiss2_mLc/1.E6<14 '+cut+')')
            h_MM2[mcsample][polarity] = r.gPad.GetPrimitive('MM2_'+name)
            startt.Draw('FitVar_El_mLc>>El_'+name+'(100,0,2600)','w_LbCorr*Event_PIDCalibEffWeight*(FitVar_El_mLc>0 && FitVar_El_mLc<2600 ' +cut+')')
            h_El[mcsample][polarity] = r.gPad.GetPrimitive('El_'+name)
            h_q2[mcsample][polarity].SetDirectory(0)
            h_MM2[mcsample][polarity].SetDirectory(0)
            h_El[mcsample][polarity].SetDirectory(0)

    h_q2_1 = PutTogetherPolarityHistos(h_q2)
    h_MM2_1 = PutTogetherPolarityHistos(h_MM2)
    h_El_1 = PutTogetherPolarityHistos(h_El)
    return h_q2_1,h_MM2_1, h_El_1

In [5]:
def PlotSuperimposed(var,h,h_TM,h_Tr,h_Presel,h_Final):
    c = r.TCanvas('c_'+var,'',800,1500)
    nsamples = len(mcsamples)
    npads = int(nsamples/2+1)
    c.Divide(2,npads)
    for i,mcsample in enumerate(mcsamples):
        c.cd(i+1)
        h[mcsample].SetLineColor(r.kBlack)
        h_TM[mcsample].SetLineColor(r.kOrange+2)
        h_Tr[mcsample].SetLineColor(r.kGreen-3)
        h_Presel[mcsample].SetLineColor(r.kAzure+9)
        h_Final[mcsample].SetLineColor(r.kRed)
        h[mcsample].Draw('hist')
        h_TM[mcsample].Draw('hist same')
        h_Tr[mcsample].Draw('hist same')
        h_Presel[mcsample].Draw('hist same')
        h_Final[mcsample].Draw('hist same')
        l = r.TLegend(0.1,0.4,0.5,0.85)
        l.AddEntry(h[mcsample],'start','l')
        l.AddEntry(h_TM[mcsample],'TruthMatched','l')
        l.AddEntry(h_Tr[mcsample],'Trigger','l')
        l.AddEntry(h_Presel[mcsample],'Presel','l')
        l.AddEntry(h_Final[mcsample],'Final Presel','l')
        l.Draw('sames')
    return c,l

In [6]:
h_q2,h_MM2, h_El = GetTemplateHistos('')
h_q2_TM,h_MM2_TM, h_El_TM = GetTemplateHistos('&&TruthMatch==1')
h_q2_Tr,h_MM2_Tr, h_El_Tr = GetTemplateHistos('&&TruthMatch==1&&Trigger==1&&MoreHLT2Sel==1')
h_q2_Presel,h_MM2_Presel, h_El_Presel = GetTemplateHistos('&&TruthMatch==1&&Trigger==1&&MoreHLT2Sel==1&&PIDCalib==1&&LcMass==1')
h_q2_Final,h_MM2_Final, h_El_Final = GetTemplateHistos('&&FinalSel==1')
h_q2_iso,h_MM2_iso, h_El_iso = GetTemplateHistos('&&FinalSel==1&&isIsolated==1')
h_q2_Kenr,h_MM2_Kenr, h_El_Kenr = GetTemplateHistos('&&FinalSel==1&&isKenriched==1')
h_q2_Lcpipi,h_MM2_Lcpipi, h_El_Lcpipi = GetTemplateHistos('&&FinalSel==1&&isLcpipi==1')

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


In [7]:
c_q2, l = PlotSuperimposed('q2',h_q2,h_q2_TM,h_q2_Tr,h_q2_Presel,h_q2_Final)
c_MM2,l = PlotSuperimposed('MM2',h_MM2,h_MM2_TM,h_MM2_Tr,h_MM2_Presel,h_MM2_Final)
c_El,l = PlotSuperimposed('El',h_El,h_El_TM,h_El_Tr,h_El_Presel,h_El_Final)

In [8]:
c_l = r.TCanvas('','',220,220)
l.Draw()
c_l.Draw()

In [9]:
c_q2.Draw()

In [10]:
c_MM2.Draw()

In [11]:
c_El.Draw()

In [12]:
def PlotTemplates(h_q2,h_MM2, h_El):
    nsamples = len(mcsamples)
    c = r.TCanvas('','',1000,1500)
    c.Divide(3,nsamples)
    for i,mcsample in enumerate(mcsamples):
        for j in range(1,4):
            n = 3*i+j
            c.cd(n)
            if j==1:
                h_q2[mcsample].Draw('hist')
            if j==2:
                h_El[mcsample].Draw('hist')
            if j==3:
                h_MM2[mcsample].Draw('hist')
    return c

c_iso = PlotTemplates(h_q2_iso,h_MM2_iso, h_El_iso)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: 


In [13]:
c_iso.Draw()

In [14]:
c_Kenr = PlotTemplates(h_q2_Kenr,h_MM2_Kenr, h_El_Kenr)
c_Kenr.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: 


In [15]:
c_Lcpipi = PlotTemplates(h_q2_Lcpipi,h_MM2_Lcpipi, h_El_Lcpipi)
c_Lcpipi.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: 


In [28]:
def GetTemplateHistoFinalFiles(category):
    h_q2 = {mcsample: {polarity: r.TH1F('h_q2_1_'+mcsample+'_'+polarity,'',40,-2,12) for polarity in polarities} for mcsample in mcsamples}
    h_MM2 = {mcsample: {polarity: r.TH1F('h_Mmiss2_1_'+mcsample+'_'+polarity,'',100,-2,12) for polarity in polarities} for mcsample in mcsamples}
    h_El = {mcsample: {polarity: r.TH1F('h_El_1_'+mcsample+'_'+polarity,'',100,0,2600) for polarity in polarities} for mcsample in mcsamples}
    for mcsample in mcsamples:
        print(mcsample)
        for polarity in polarities:
            f = r.TFile(folder+mcsample+'_'+polarity+'_full_preselected_'+category+'_LbCorr.root','READ')
            t = f.Get('DecayTree')
            name = mcsample+'_'+polarity
            t.Draw('FitVar_q2_mLc/1.E6>>q2_'+name+'(40,-2,14)','w_LbCorr*Event_PIDCalibEffWeight*(FitVar_q2_mLc/1.E6>-2 && FitVar_q2_mLc/1.E6<14)')
            h_q2[mcsample][polarity] = r.gPad.GetPrimitive('q2_'+name)
            t.Draw('FitVar_Mmiss2_mLc/1.E6>>MM2_'+name+'(100,-2,14)','w_LbCorr*Event_PIDCalibEffWeight*(FitVar_Mmiss2_mLc/1.E6>-2 && FitVar_Mmiss2_mLc/1.E6<14)')
            h_MM2[mcsample][polarity] = r.gPad.GetPrimitive('MM2_'+name)
            t.Draw('FitVar_El_mLc>>El_'+name+'(100,0,2600)','w_LbCorr*Event_PIDCalibEffWeight*(FitVar_El_mLc>0 && FitVar_El_mLc<2600)')
            h_El[mcsample][polarity] = r.gPad.GetPrimitive('El_'+name)
            if(h_q2[mcsample][polarity]):
                h_q2[mcsample][polarity].SetDirectory(0)
            if(h_MM2[mcsample][polarity]):
                h_MM2[mcsample][polarity].SetDirectory(0)
            if(h_El[mcsample][polarity]):
                h_El[mcsample][polarity].SetDirectory(0)

    h_q2_1 = PutTogetherPolarityHistos(h_q2)
    h_MM2_1 = PutTogetherPolarityHistos(h_MM2)
    h_El_1 = PutTogetherPolarityHistos(h_El)
    return h_q2_1,h_MM2_1, h_El_1

In [32]:
h_q2_iso_1, h_MM2_iso_1, h_El_iso_1 = GetTemplateHistoFinalFiles('iso')
h_q2_Kenr_1, h_MM2_Kenr_1, h_El_Kenr_1 = GetTemplateHistoFinalFiles('Kenr')
h_q2_Lcpipi_1, h_MM2_Lcpipi_1, h_El_Lcpipi_1 = GetTemplateHistoFinalFiles('Lcpipi')

Lb_Lcmunu
Lb_Lctaunu
Lb_LcDs
Lb_Lc2593munu
Lb_Lc2593taunu
Lb_Lc2593Ds
Lb_Lc2625munu
Lb_Lc2625taunu
Lb_Lc2625Ds
Lb_Lcmunu
Lb_Lctaunu
Lb_LcDs
Lb_Lc2593munu
Lb_Lc2593taunu
Lb_Lc2593Ds
Lb_Lc2625munu
Lb_Lc2625taunu
Lb_Lc2625Ds
Lb_Lcmunu
Lb_Lctaunu
Lb_LcDs
Lb_Lc2593munu
Lb_Lc2593taunu
Lb_Lc2593Ds
Lb_Lc2625munu
Lb_Lc2625taunu
Lb_Lc2625Ds


In [33]:
def PlotSuperimposedTemplates(h_q2,h_MM2, h_El,h_q2_1,h_MM2_1, h_El_1):
    nsamples = len(mcsamples)
    c = r.TCanvas('','',1000,1500)
    c.Divide(3,nsamples)
    for i,mcsample in enumerate(mcsamples):
        h_q2_1[mcsample].SetLineColor(r.kOrange+2)
        h_MM2_1[mcsample].SetLineColor(r.kOrange+2)
        h_El_1[mcsample].SetLineColor(r.kOrange+2)
        for j in range(1,4):
            n = 3*i+j
            c.cd(n)
            if j==1:
                h_q2[mcsample].Draw('hist')
                h_q2_1[mcsample].Draw('hist sames')
            if j==2:
                h_El[mcsample].Draw('hist')
                h_El_1[mcsample].Draw('hist sames')
            if j==3:
                h_MM2[mcsample].Draw('hist')
                h_MM2_1[mcsample].Draw('hist sames')
    return c

In [34]:
c_iso_1 = PlotSuperimposedTemplates(h_q2_iso,h_MM2_iso, h_El_iso,h_q2_iso_1,h_MM2_iso_1, h_El_iso_1)
c_iso_1.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: 


In [35]:
c_Kenr_1 = PlotSuperimposedTemplates(h_q2_Kenr,h_MM2_Kenr, h_El_Kenr,h_q2_Kenr_1,h_MM2_Kenr_1, h_El_Kenr_1)
c_Kenr_1.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: 


In [37]:
c_Lcpipi_1 = PlotSuperimposedTemplates(h_q2_Lcpipi,h_MM2_Lcpipi, h_El_Lcpipi,h_q2_Lcpipi_1,h_MM2_Lcpipi_1, h_El_Lcpipi_1)
c_Lcpipi_1.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: 


In [ ]:
def GetHistosForCorrectionCheck(category,var,nbins,bmin,bmax):
    h = {mcsample: {polarity: r.TH1F('h_'+var+'_'+mcsample+'_'+polarity,'',nbins,bmin,bmax) for polarity in polarities} for mcsample in mcsamples}
    for mcsample in mcsamples:
        for polarity in polarities:
            f = r.TFile(folder+mcsample+'_'+polarity+'_full_preselected_'+category+'_LbCorr.root','READ')
            t = f.Get('DecayTree')
            name = mcsample+'_'+polarity
            t.Draw(var+'>>'+var+'_'+name+'('+str(nbins)+','+str(bmin)+','+str(bmax)+')',
                   'w_LbCorr*Event_PIDCalibEffWeight*('+var+'>'+str(bmin)+'&&'+var+'<'+str(bmax)+')')
            h[mcsample][polarity] = r.gPad.GetPrimitive(var+'_'+name)
            h[mcsample][polarity].SetDirectory(0)
    h_1 = PutTogetherPolarityHistos(h)
    return h_1

In [ ]:
var = {'P':['Lb_P',[40,0,6e5]],'Pt':['Lb_PT',[100,0,4e4]],
       'mu_Pt':['mu_PT',[40,0,3e4]],"IPCH2":["Lb_IPCHI2_OWNPV",[100,0,5e3]],
       "vtx2":["Lb_ENDVERTEX_CHI2",[100,0,6]]}

In [ ]:
hMC = {}
for iv in variables.keys():
    hMC[iv] = GetHistosForCorrectionCheck('iso',var[iv][0],var[iv][1][0],var[iv][1][1],var[iv][1][2])